In [1]:
import sys
sys.path.append('/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/dhnlplotting/util')

import glob
import os
from plot_base import PlotBase
from IPython.display import display, Image
import ROOT
ROOT.gROOT.SetStyle("ATLAS")
from array import array
import importlib
import plot_base
import plot_util
import numpy as np
importlib.reload(plot_base)
importlib.reload(plot_util)



Welcome to JupyROOT 6.22/06


<module 'plot_util' from '/home/newhouse/public/Analysis/HNL/dhnlanalysisnotebooks/systematics/dhnlplotting/util/plot_util.py'>